In [2]:
import pandas as pd

train_mobile = pd.read_csv('train_data_mobile.csv')
train_static = pd.read_csv('train_data_static.csv')
test_data = pd.read_csv('test_data.csv')

In [3]:
train_data_table = pd.concat([train_mobile, train_static])

In [4]:
train_data_table

,time,hmd,spd,tmp,pm2d5,lat,lon
0,2018-10-10 18:00:01,72.60,11.54,23.10,62.50,23.047180,113.149890
1,2018-10-10 18:00:04,72.57,10.65,23.10,62.00,23.047130,113.149900
2,2018-10-10 18:00:07,72.50,14.65,23.10,62.67,23.047000,113.149920
3,2018-10-10 18:00:10,72.50,17.50,23.10,62.33,23.046860,113.149920
4,2018-10-10 18:00:13,72.47,6.80,23.10,60.67,23.046800,113.149920
...,...,...,...,...,...,...,...
430758,2018-10-13 17:59:47,44.90,0.10,30.00,51.00,23.063979,113.139616
430759,2018-10-13 17:59:50,44.90,0.22,30.00,50.33,23.063979,113.139616
430760,2018-10-13 17:59:53,44.90,0.10,30.00,50.00,23.063979,113.139616
430761,2018-10-13 17:59:56,44.90,0.19,30.03,49.67,23.063979,113.139616


In [5]:
train_data_table = train_data_table.reset_index(drop=True)

In [6]:
train_data_cell = []
count = []
flag = 0
for i in range(1,train_data_table.shape[0]):
    if train_data_table['time'][i] < train_data_table['time'][i-1]:
        count.append(i)
        if i > train_static.shape[0] and flag == 0:
            num_static = len(count)-1
            flag = 1
count = [0] + count + [train_data_table.shape[0]]
print(count)
for i in range(1, len(count)):
    train_data_cell.append(train_data_table.iloc[count[i-1]:count[i], :])
num_mobile = len(train_data_cell) - num_static
data_combine = train_data_cell

[0, 37759, 82356, 124202, 165572, 224859, 283571, 323654, 378030, 464130, 550338, 636483, 722709, 808793]


In [7]:
data_combine[0]

,time,hmd,spd,tmp,pm2d5,lat,lon
0,2018-10-10 18:00:01,72.60,11.54,23.1,62.50,23.04718,113.14989
1,2018-10-10 18:00:04,72.57,10.65,23.1,62.00,23.04713,113.14990
2,2018-10-10 18:00:07,72.50,14.65,23.1,62.67,23.04700,113.14992
3,2018-10-10 18:00:10,72.50,17.50,23.1,62.33,23.04686,113.14992
4,2018-10-10 18:00:13,72.47,6.80,23.1,60.67,23.04680,113.14992
...,...,...,...,...,...,...,...
37754,2018-10-13 17:59:48,45.80,0.18,30.3,45.33,23.05404,113.13995
37755,2018-10-13 17:59:51,45.80,0.23,30.3,45.00,23.05404,113.13995
37756,2018-10-13 17:59:54,45.80,0.30,30.3,45.33,23.05404,113.13995
37757,2018-10-13 17:59:57,45.80,0.26,30.3,46.00,23.05403,113.13994


In [6]:
num_static,num_mobile

(8, 5)

In [7]:
len(data_combine)

13

In [8]:
import numpy as np

data_total_minute = [0]*len(data_combine)
for i in range(len(data_combine)):
    time = pd.to_datetime(data_combine[i]['time'])
    date = time.dt.date
    unique_date = date.unique()
    date_ = time.dt.strftime('%m/%d/%Y')
    dayName = time.dt.strftime('%A')
    hour_ = time.dt.strftime('%H')
    minute_ = time.dt.strftime('%M')
    dayNumber = time.dt.weekday
    data = data_combine[i].assign(date_=date_, dayName=dayName, hour_=hour_, minute_=minute_, dayNumber=dayNumber)
    data_gb_datemin = data.groupby(['date_', 'hour_', 'minute_']).mean()
    time_minutely = pd.to_datetime(date_ + ' ' + hour_ + ':' + minute_, format='%m/%d/%Y %H:%M')
    time_minutely = np.unique(time_minutely)
    hmd = data_gb_datemin['hmd']
    spd = data_gb_datemin['spd']
    tmp = data_gb_datemin['tmp']
    pm2d5 = data_gb_datemin['pm2d5']
    lat = data_gb_datemin['lat']
    lon = data_gb_datemin['lon']
    data_total_minute[i] = pd.DataFrame({'time':time_minutely, 'hmd':hmd, 'spd':spd, 'tmp':tmp, 'pm2d5':pm2d5, 'lat':lat, 'lon':lon})
    # data_total_minute[i] = data_total_minute[i].rename(columns={0: "time", 1:'hmd', 2:'spd', 3:'tmp', 4:'pm2d5', 5:'lat', 6:'lon'})


In [112]:
data_total_minute

[                                        time        hmd        spd        tmp  \
 date_      hour_ minute_                                                        
 10/10/2018 18    00      2018-10-10 18:00:00  72.269000   9.322500  23.141500   
                  01      2018-10-10 18:01:00  72.040000  10.764500  23.216500   
                  02      2018-10-10 18:02:00  71.694500  12.371000  23.300000   
                  03      2018-10-10 18:03:00  71.566316  19.153684  23.392632   
                  04      2018-10-10 18:04:00  71.405500  19.408500  23.458500   
 ...                                      ...        ...        ...        ...   
 10/13/2018 17    56      2018-10-13 17:56:00  46.329500   0.140500  30.200000   
                  57      2018-10-13 17:57:00  45.988000   0.155500  30.200000   
                  58      2018-10-13 17:58:00  45.838500   0.138000  30.271000   
                  59      2018-10-13 17:59:00  45.916500   0.170000  30.300000   
            18  

In [9]:
  scale = 1
  scale_parameter = 1

In [10]:
# seasonal decomposition for static data
data_cleaned_static = []
for i in range(num_static):  # only static
    T = 60 * 24 * scale
    sw25 = [1/(2*T)] + [1/T] * (T - 1) + [1/(2*T)]
    trend = np.convolve(data_total_minute[i]['pm2d5'], sw25, 'same')

    # padding the trend
    trend_padded = np.pad(trend[int(T/2):-int(T/2)], (int(T/2), int(T/2)), "constant", constant_values=(trend[int(T/2)], trend[-int(T/2)]))

    T1 = len(data_total_minute[i]['pm2d5'])
    sidx = {}
    for ii in range(T):
        sidx[ii] = list(range(ii, T1, T))

    detrend_data = data_total_minute[i]['pm2d5'] - trend_padded  # detrend
    seasonal = [np.mean(detrend_data[sidx[ii]]) for ii in range(T)]  # generate seasonal trend
    nc = T1 // T
    rm = T1 % T
    seasonal = seasonal * nc + seasonal[:rm]
    sBar = np.mean(seasonal)
    seasonal = seasonal - sBar
    residual = data_total_minute[i]['pm2d5'] - trend_padded - seasonal
    pm2d5_clean = trend_padded + seasonal
    time = data_total_minute[i]['time']
    data_cleaned_static.append(pd.DataFrame({'time': time, 'pm2d5_clean': pm2d5_clean}))

In [11]:
# seasonal decomposition for mobile
data_cleaned_mobile = []
for i in range(num_static,num_static+num_mobile):  # only mobile
    T = 60 * 24 * scale
    sw25 = [1/(2*T)] + [1/T] * (T - 1) + [1/(2*T)]
    trend = np.convolve(data_total_minute[i]['pm2d5'], sw25, 'same')

    # padding the trend
    trend_padded = np.pad(trend[int(T/2):-int(T/2)], (int(T/2), int(T/2)), "constant", constant_values=(trend[int(T/2)], trend[-int(T/2)]))

    T1 = len(data_total_minute[i]['pm2d5'])
    sidx = {}
    for ii in range(T):
        sidx[ii] = list(range(ii, T1, T))

    detrend_data = data_total_minute[i]['pm2d5'] - trend_padded  # detrend
    seasonal = [np.mean(detrend_data[sidx[ii]]) for ii in range(T)]  # generate seasonal trend
    nc = T1 // T
    rm = T1 % T
    seasonal = seasonal * nc + seasonal[:rm]
    sBar = np.mean(seasonal)
    seasonal = seasonal - sBar
    residual = data_total_minute[i]['pm2d5'] - trend_padded - seasonal
    pm2d5_clean = trend_padded + seasonal
    time = data_total_minute[i]['time']
    data_cleaned_mobile.append(pd.DataFrame({'time': time, 'pm2d5_clean': pm2d5_clean}))

In [125]:
data_cleaned_mobile

[                                        time  pm2d5_clean
 date_      hour_ minute_                                 
 10/10/2018 18    00      2018-10-10 18:00:00    38.270545
                  01      2018-10-10 18:01:00    37.460145
                  02      2018-10-10 18:02:00    37.939164
                  03      2018-10-10 18:03:00    38.434178
                  04      2018-10-10 18:04:00    38.029924
 ...                                      ...          ...
 10/13/2018 17    55      2018-10-13 17:55:00    43.442850
                  56      2018-10-13 17:56:00    42.459776
                  57      2018-10-13 17:57:00    42.449898
                  58      2018-10-13 17:58:00    41.795777
                  59      2018-10-13 17:59:00    42.307358
 
 [4320 rows x 2 columns],
                                         time  pm2d5_clean
 date_      hour_ minute_                                 
 10/10/2018 18    00      2018-10-10 18:00:00    36.172245
                  01      20

In [12]:
from scipy.signal import gaussian

def smooth_data(data, window_size):
    # Generate a 1D Gaussian kernel
    kernel = gaussian(window_size, std=window_size / 4)
    # Normalize the kernel
    kernel /= kernel.sum()
    # Convolve the data with the kernel
    smooth_data = np.convolve(data, kernel, 'same')
    return smooth_data
window_size = 60*24*scale_parameter

data_total_pro = data_total_minute
for i in range(num_static):
    data_total_pro[i]['pm2d5'] = data_cleaned_static[i]['pm2d5_clean']
    data_total_pro[i]['hmd'] = smooth_data(data_total_pro[i]['hmd'], window_size)
    data_total_pro[i]['spd'] = smooth_data(data_total_pro[i]['spd'], window_size)
    data_total_pro[i]['tmp'] = smooth_data(data_total_pro[i]['tmp'], window_size)

In [13]:
#parameter cleaning for mobile sensor
for i in range(num_mobile):
    data_total_pro[i+num_static]['pm2d5'] = data_cleaned_mobile[i]['pm2d5_clean']
    data_total_pro[i+num_static]['hmd'] = smooth_data(data_total_pro[i+num_static]['hmd'], window_size)
    data_total_pro[i+num_static]['spd'] = smooth_data(data_total_pro[i+num_static]['spd'], window_size)
    data_total_pro[i+num_static]['tmp'] = smooth_data(data_total_pro[i+num_static]['tmp'], window_size)

In [200]:
import pandas as pd

#putting data into one matrix and sort the data
data_onematrix = pd.concat(data_total_pro)
data_total_sorted = data_onematrix.sort_values(by='time')

#normalize the data, using range
timenum_train = pd.to_datetime(data_total_sorted['time']).astype(int) // 10**9
test_data['time'] = pd.to_datetime(test_data['time']).astype(int) // 10**9
data_total_norm_para = data_total_sorted[['time', 'hmd', 'spd', 'tmp', 'lat', 'lon']]
# data_total_norm_para_withtest = pd.concat([data_total_norm_para, test_data])
# data_total_norm_para_withtest = (data_total_norm_para_withtest - data_total_norm_para_withtest.min()) / (data_total_norm_para_withtest.max() - data_total_norm_para_withtest.min())
data_total_norm_para = (data_total_norm_para - data_total_norm_para.min()) / (data_total_norm_para.max() - data_total_norm_para.min())

<ipython-input-200-4dd9c1467c53>:8: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  timenum_train = pd.to_datetime(data_total_sorted['time']).astype(int) // 10**9
<ipython-input-200-4dd9c1467c53>:9: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  test_data['time'] = pd.to_datetime(test_data['time']).astype(int) // 10**9


In [209]:
x = data_total_norm_para
y = data_total_sorted['pm2d5']

In [202]:
x.shape[0]

40916

In [210]:
x = x.interpolate()

In [204]:
x

time       hmd       spd       tmp       lat  \
date_      hour_ minute_                                                     
10/10/2018 18    00       0.000000  0.140802  0.419463  0.027903  0.348846   
                 00       0.000000   0.13576  0.364544  0.020971  0.530174   
                 00       0.000000    0.1724  0.358073  0.027009  0.551273   
                 00       0.000000  0.160968  0.326464   0.02243  0.208987   
                 00       0.000000       0.0   0.00207  0.146216  0.341385   
...                            ...       ...       ...       ...       ...   
10/13/2018 17    59       0.999769  0.073084  0.002129  0.121427  0.501639   
           18    00       1.000000  0.080786  0.487711   0.09991  0.415022   
                 00       1.000000  0.059352  0.000084  0.116717  0.390836   
                 00       1.000000  0.074076  0.469015  0.097812  0.513227   
                 00       1.000000  0.096046  0.278206  0.091844  0.451152   

                               lon  
date_      hour_ minute_            
10/10/2018 18    00       0.854722  
                 00       0.923844  
                 00       0.839368  
                 00       0.802972  
                 00       0.924303  
...                            ...  
10/13/2018 17    59        0.80672  
           18    00       0.808231  
                 00       0.740145  
                 00       0.828103  
                 00       0.765549  

[40916 rows x 6 columns]

In [211]:
y = y.interpolate()

In [35]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
kfold = KFold(n_splits=5)
model = RandomForestRegressor(random_state=42)
scoring = 'neg_mean_squared_error'
results = cross_val_score(model, x, y, cv=kfold, scoring=scoring)

#calculate and print the mean and standard deviation of the RMSE
rmse_scores = np.sqrt(-results)
print(f"RMSE scores: {rmse_scores}")

RMSE scores: [13.13360772 16.43253044 14.80278617 16.49591012 21.22524687]


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# define the Matern/Exponential kernel function
kernel = Matern()

# create the GPR model with the Matern/Exponential kernel function
gpr = GaussianProcessRegressor(kernel=kernel)
gpr.fit(x, y)

scores = -1 * cross_val_score(gpr, x, y, cv=kfold, scoring=scoring)
rmse = np.sqrt(np.mean(scores))

print(f'RMSE: {rmse:.3f}')

In [212]:
x = x.values
x.reshape(-1,6)

array([[0.0, 0.14080231282302116, 0.4194630653445531,
        0.02790342453651101, 0.34884643607824223, 0.854722204729555],
       [0.0, 0.1357603573448097, 0.3645437884404367, 0.02097109400610225,
        0.5301736253622158, 0.9238439094019925],
       [0.0, 0.17239972666322373, 0.3580731538943253,
        0.027009124002266256, 0.551272962451619, 0.839368304680327],
       ...,
       [1.0, 0.05935244993586504, 8.354261917783823e-05,
        0.11671728101358511, 0.3908361628297467, 0.740145212489289],
       [1.0, 0.07407576000851135, 0.46901505513453073,
        0.09781155368736871, 0.5132271183910921, 0.8281030248325864],
       [1.0, 0.09604567334698032, 0.27820569074963714,
        0.0918435700364906, 0.4511523201870452, 0.7655492348706086]],
      dtype=object)

In [213]:
x.shape[0]

40916

In [214]:
x = np.reshape(x, (-1, 1, 6))

In [215]:
y = y.values
y = np.reshape(y, (-1, 1, 1))

In [235]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

x = tf.convert_to_tensor(x, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)

# define the model
model = tf.keras.Sequential()
model.add(LSTM(32, input_shape=(1, 6), activation='relu'))
model.add(Dense(1))

# compile the model with Adam optimization and RMSE loss function
model.compile(optimizer=Adam(learning_rate=0.008), loss='mean_squared_error')

# fit the model to the data
history = model.fit(x, y, epochs=10, batch_size=64, validation_split=0.2)

# use cross-validation to evaluate the model
scores = model.evaluate(x, y, verbose=0)
print("RMSE:", np.sqrt(scores))

Epoch 1/10
512/512 [==============================] - 2s 2ms/step - loss: 263.6782 - val_loss: 184.7177
Epoch 2/10
512/512 [==============================] - 1s 2ms/step - loss: 108.9625 - val_loss: 146.2929
Epoch 3/10
512/512 [==============================] - 1s 2ms/step - loss: 101.0138 - val_loss: 147.7546
Epoch 4/10
512/512 [==============================] - 1s 2ms/step - loss: 99.5098 - val_loss: 163.9632
Epoch 5/10
512/512 [==============================] - 1s 2ms/step - loss: 98.9752 - val_loss: 161.9338
Epoch 6/10
512/512 [==============================] - 1s 2ms/step - loss: 98.5144 - val_loss: 161.2200
Epoch 7/10
512/512 [==============================] - 1s 2ms/step - loss: 98.0421 - val_loss: 147.5322
Epoch 8/10
512/512 [==============================] - 1s 2ms/step - loss: 95.3189 - val_loss: 145.1168
Epoch 9/10
512/512 [==============================] - 1s 2ms/step - loss: 90.1263 - val_loss: 137.3894
Epoch 10/10
512/512 [==============================] - 1s 2ms/step - l